# Transformations before model training

In [ ]:
# Import libraries

import pyspark.pandas as ps
from pyspark.ml.feature import StringIndexer, OneHotEncoder
import numpy as np

StatementMeta(, 2ceb281d-ec00-49c2-8434-b36273ac882e, 31, Finished, Available)

In [30]:
psdf = ps.DataFrame(spark.sql("SELECT * FROM sb_lakehouse.airbnb_listings_loaded"))

StatementMeta(, 2ceb281d-ec00-49c2-8434-b36273ac882e, 32, Finished, Available)

In [31]:
psdf.head(10)

StatementMeta(, 2ceb281d-ec00-49c2-8434-b36273ac882e, 33, Finished, Available)

,access,accommodates,amenities,availability_30,availability_365,availability_60,availability_90,bathrooms,bed_type,bedrooms,beds,calculated_host_listings_count,calendar_last_scraped,calendar_updated,cancellation_policy,city,cleaning_fee,country,country_code,description,experiences_offered,extra_people,features,first_review,geolocation,guests_included,host_about,host_id,host_listings_count,host_location,host_name,host_neighbourhood,host_picture_url,host_response_rate,host_response_time,host_since,host_thumbnail_url,host_total_listings_count,host_url,host_verifications,house_rules,id,interaction,last_review,last_scraped,latitude,listing_url,longitude,market,maximum_nights,medium_url,minimum_nights,monthly_price,name,neighborhood_overview,neighbourhood,neighbourhood_cleansed,notes,number_of_reviews,picture_url,price,property_type,review_scores_accuracy,review_scores_checkin,review_scores_cleanliness,review_scores_communication,review_scores_location,review_scores_rating,review_scores_value,reviews_per_month,room_type,scrape_id,security_deposit,smart_location,space,square_feet,state,street,summary,thumbnail_url,transit,weekly_price,xl_picture_url,zipcode
0,Located in the center of Rome but far from the...,4,"TV,Internet,Wireless Internet,Wheelchair acces...",0,312,7,37,1.0,Real Bed,1,2,1,2017-05-09,3 months ago,flexible_new,roma\nroma,NaN,Italy,IT,Grazioso appartamento con terazzo ben collegat...,none,8,"Host Has Profile Pic,Host Identity Verified",2015-05-12,"[41.88147895082083, 12.520587717952644]",2,None,18791174,1,"Rome, Lazio, Italy",Antonio,None,https://a0.muscache.com/im/users/18791174/prof...,80,within an hour,2014-07-24,https://a0.muscache.com/im/users/18791174/prof...,1,https://www.airbnb.com/users/show/18791174,"email,phone,reviews,jumio",Essere educati con i vicini di casa e non crea...,6210987,"Consigli e aiuti per un piacevole soggiorno, e...",2016-12-15,2017-05-09,41.88147895082083,https://www.airbnb.com/rooms/6210987,12.520587717952644,Rome,1125,https://a0.muscache.com/im/pictures/77517183/6...,2,920.0,Centro di Roma ottimi collegamenti,"Quartiere ottimo per fare shopping, pieno di s...",None,VII San Giovanni/Cinecittà,"Cucina avere tutto il necessario per cucinare,...",18,"{'color_summary': ['rgba(58, 55, 54, 1.00)', '...",65,Apartment,9,9,9,10,9,91,9,0.74,Entire home/apt,20170508161515,150.0,"roma\nroma, Italy",Silenzioso appartamento formato da una grande ...,NaN,Lazio,"roma\nroma, Lazio 00182, Italy",Grazioso appartamento con terazzo ben collegat...,https://a0.muscache.com/im/pictures/77517183/6...,Con la metro a 200metri si raggiunge tutta rom...,400.0,https://a0.muscache.com/im/pictures/77517183/6...,00182
1,The apartment is located on the first floor an...,2,"TV,Internet,Wireless Internet,Air conditioning...",0,250,11,14,1.0,Real Bed,1,1,1,2017-05-09,a week ago,moderate_new,Roma,30.0,Italy,IT,Special offer 15% off: 76 € / night instead of...,none,0,"Host Is Superhost,Host Has Profile Pic,Host Id...",2014-09-09,"[41.91263353238262, 12.452129763283953]",2,I am a traveller and love to know other cultur...,4068282,1,"Rome, Lazio, Italy",Angelo,Prati,https://a0.muscache.com/im/pictures/eb390a18-f...,100,within a few hours,2012-11-05,https://a0.muscache.com/im/pictures/eb390a18-f...,1,https://www.airbnb.com/users/show/4068282,"email,phone,manual_online,reviews,manual_offli...",The apartment is located in a condominium and ...,3013955,I will be glad to welcome you on your arrival ...,2017-05-04,2017-05-09,41.91263353238262,https://www.airbnb.com/rooms/3013955,12.452129763283953,Rome,1125,https://a0.muscache.com/im/pictures/45641230/0...,3,1650.0,Beautiful new apt close to St Peter,The apartment is located in the Prati district...,Prati,I Centro Storico,Minimum stay of two nights. City tax is requir...,39,"{'color_summary': ['rgba(232, 234, 228, 1.00)'...",79,Apartment,10,10,10,10,9,97,10,1.20,Entire home/apt,20170508161515,150.0,"Roma, Italy",Beautiful newly renovated apartment. Bright an...,NaN,Lazio,"Prati, Roma,

### Determine which columns to include

There are a lot of columns in this dataframe (89). Price will be our y-column. Let's determine which columns we are going to select for our first model:

My first model is going to be fairly simple, I am not going to use NLP to create columns out of text columns. Just numerical columns and categorical columns



In [32]:
psdf.isna().sum()


StatementMeta(, 2ceb281d-ec00-49c2-8434-b36273ac882e, 34, Finished, Available)

/opt/spark/python/lib/pyspark.zip/pyspark/pandas/internal.py:1573: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  fields = [
/opt/spark/python/lib/pyspark.zip/pyspark/sql/pandas/conversion.py:604: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  [(c, t) for (_, c), t in zip(pdf_slice.iteritems(), arrow_types)]


access                             78357
accommodates                           0
amenities                           1290
availability_30                        0
availability_365                       0
availability_60                        0
availability_90                        0
bathrooms                            363
bed_type                               0
bedrooms                             215
beds                                 306
calculated_host_listings_count         0
calendar_last_scraped                  0
calendar_updated                       0
cancellation_policy                    0
city                                   0
cleaning_fee                       62104
country                                0
country_code                           0
description                           55
experiences_offered                    0
extra_people                          17
features                             118
first_review                       42950
geolocation     

### 

Based on the EDA and some basic domain knowledge we choose to select following columns

In [ ]:
columns_to_select =['accommodates', 'availability_30',
       'availability_365', 'availability_60', 'availability_90', 'bathrooms',
       'bed_type', 'bedrooms', 'beds', 
        'cancellation_policy','city', 'extra_people', 'features', 'first_review',
        'guests_included', 
        'host_listings_count',
       'host_response_rate', 'host_response_time', 'host_since',
        'host_total_listings_count', 
       'host_verifications',  'last_review', 'latitude',
        'longitude', 'maximum_nights',
        'minimum_nights', 
         'number_of_reviews',
        'price', 'property_type', 'review_scores_accuracy',
       'review_scores_checkin', 'review_scores_cleanliness',
       'review_scores_communication', 'review_scores_location',
       'review_scores_rating', 'review_scores_value', 'reviews_per_month',
       'room_type']


StatementMeta(, 2ceb281d-ec00-49c2-8434-b36273ac882e, 36, Finished, Available)

In [35]:
column_subset = psdf[psdf.columns.intersection(columns_to_select)]

StatementMeta(, 2ceb281d-ec00-49c2-8434-b36273ac882e, 37, Finished, Available)

In [36]:
column_subset.head(3)

StatementMeta(, 2ceb281d-ec00-49c2-8434-b36273ac882e, 38, Finished, Available)

,accommodates,availability_30,availability_365,availability_60,availability_90,bathrooms,bed_type,bedrooms,beds,cancellation_policy,city,extra_people,features,first_review,guests_included,host_listings_count,host_response_rate,host_response_time,host_since,host_total_listings_count,host_verifications,last_review,latitude,longitude,maximum_nights,minimum_nights,number_of_reviews,price,property_type,review_scores_accuracy,review_scores_checkin,review_scores_cleanliness,review_scores_communication,review_scores_location,review_scores_rating,review_scores_value,reviews_per_month,room_type
0,4,0,312,7,37,1.0,Real Bed,1,2,flexible_new,roma\nroma,8,"Host Has Profile Pic,Host Identity Verified",2015-05-12,2,1,80,within an hour,2014-07-24,1,"email,phone,reviews,jumio",2016-12-15,41.88147895082083,12.520587717952644,1125,2,18,65,Apartment,9,9,9,10,9,91,9,0.74,Entire home/apt
1,2,0,250,11,14,1.0,Real Bed,1,1,moderate_new,Roma,0,"Host Is Superhost,Host Has Profile Pic,Host Id...",2014-09-09,2,1,100,within a few hours,2012-11-05,1,"email,phone,manual_online,reviews,manual_offli...",2017-05-04,41.91263353238262,12.452129763283953,1125,3,39,79,Apartment,10,10,10,10,9,97,10,1.20,Entire home/apt
2,8,2,280,5,16,2.0,Real Bed,3,4,flexible_new,Roma,0,"Host Has Profile Pic,Host Identity Verified,Is...",2017-02-16,1,67,99,within an hour,2013-11-23,67,"email,phone,reviews,jumio,government_id",2017-02-16,41.89707107334837,12.476088931362893,1125,3,1,350,Apartment,10,10,10,10,10,100,8,0.36,Entire home/apt


### Feature engineering

The following columns have to be one-hot encoded:

- bed type
- cancelation_policy
- city
- host_response_time
- property type
- room type

The following values should also be one-hot encoded, only they are now not in category format but plain text:

- features
- host verification

Convert date variables to int -> days untill 3 Oct 2017:

- first_review
- host-since
- last review

Calculate polar coordinates

- By using longitude and latitude

Also:

- mean inpute empty values

In [37]:
ps_encoded_a = ps.get_dummies(column_subset, columns = ["bed_type", "cancellation_policy", 
                                                        "host_response_time", "property_type", "room_type"])\
                                                        .drop(["city", "bed_type", "cancellation_policy", 
                                                        "host_response_time", "property_type", "room_type"], axis = 1)

StatementMeta(, 2ceb281d-ec00-49c2-8434-b36273ac882e, 39, Finished, Available)

In [38]:
ps_encoded_a.head(3)

StatementMeta(, 2ceb281d-ec00-49c2-8434-b36273ac882e, 40, Finished, Available)

,accommodates,availability_30,availability_365,availability_60,availability_90,bathrooms,bedrooms,beds,extra_people,features,first_review,guests_included,host_listings_count,host_response_rate,host_since,host_total_listings_count,host_verifications,last_review,latitude,longitude,maximum_nights,minimum_nights,number_of_reviews,price,review_scores_accuracy,review_scores_checkin,review_scores_cleanliness,review_scores_communication,review_scores_location,review_scores_rating,review_scores_value,reviews_per_month,bed_type_Airbed,bed_type_Couch,bed_type_Futon,bed_type_Pull-out Sofa,bed_type_Real Bed,cancellation_policy_flexible,cancellation_policy_flexible_new,cancellation_policy_moderate,cancellation_policy_moderate_new,cancellation_policy_strict,cancellation_policy_strict_new,cancellation_policy_super_strict_30,cancellation_policy_super_strict_30_new,cancellation_policy_super_strict_60,cancellation_policy_super_strict_60_new,host_response_time_a few days or more,host_response_time_within a day,host_response_time_within a few hours,host_response_time_within an hour,property_type_Apartment,property_type_Bed & Breakfast,property_type_Boat,property_type_Boutique hotel,property_type_Bungalow,property_type_Cabin,property_type_Camper/RV,property_type_Car,property_type_Casa particular,property_type_Castle,property_type_Cave,property_type_Chalet,property_type_Condominium,property_type_Dorm,property_type_Earth House,property_type_Guest suite,property_type_Guesthouse,property_type_Hostel,property_type_House,property_type_Hut,property_type_In-law,property_type_Island,property_type_Lighthouse,property_type_Loft,property_type_Nature lodge,property_type_Other,property_type_Parking Space,property_type_Pension (Korea),property_type_Plane,property_type_Ryokan (Japan),property_type_Serviced apartment,property_type_Tent,property_type_Timeshare,property_type_Tipi,property_type_Townhouse,property_type_Train,property_type_Treehouse,property_type_Vacation home,property_type_Villa,property_type_Yurt,room_type_Entire home/apt,room_type_Private room,room_type_Shared room
0,4,0,312,7,37,1.0,1,2,8,"Host Has Profile Pic,Host Identity Verified",2015-05-12,2,1,80,2014-07-24,1,"email,phone,reviews,jumio",2016-12-15,41.88147895082083,12.520587717952644,1125,2,18,65,9,9,9,10,9,91,9,0.74,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
1,2,0,250,11,14,1.0,1,1,0,"Host Is Superhost,Host Has Profile Pic,Host Id...",2014-09-09,2,1,100,2012-11-05,1,"email,phone,manual_online,reviews,manual_offli...",2017-05-04,41.91263353238262,12.452129763283953,1125,3,39,79,10,10,10,10,9,97,10,1.20,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
2,8,2,280,5,16,2.0,3,4,0,"Host Has Profile Pic,Host Identity Verified,Is...",2017-02-16,1,67,99,2013-11-23,67,"email,phone,reviews,jumio,government_id",2017-02-16,41.89707107334837,12.476088931362893,1125,3,1,350,10,10,10,10,10,100,8,0.36,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0


In [39]:
# Convert columns to datetime
import pyspark.sql

constant_date = ps.to_datetime('2017-10-03')

datetime_columns = ['first_review', 'last_review', 'host_since']

for col in datetime_columns:
    ps_encoded_a[col] = ps.to_datetime(ps_encoded_a[col])

StatementMeta(, 2ceb281d-ec00-49c2-8434-b36273ac882e, 41, Finished, Available)

In [40]:
from pyspark.sql.types import StructType, StructField, StringType

sdf = ps_encoded_a.to_spark()

StatementMeta(, 2ceb281d-ec00-49c2-8434-b36273ac882e, 42, Finished, Available)

/opt/spark/python/lib/pyspark.zip/pyspark/pandas/utils.py:975: PandasAPIOnSparkAdviceWarning: If `index_col` is not specified for `to_spark`, the existing index is lost when converting to Spark DataFrame.
  warnings.warn(message, PandasAPIOnSparkAdviceWarning)


In [41]:
from pyspark.sql.functions import datediff, lit, col
from datetime import datetime

constant_date = lit("2017-10-03")

# Calculate the difference in days between the date column and the constant date
for column in datetime_columns:
    sdf = sdf.withColumn(column,  datediff(constant_date, col(column)))

StatementMeta(, 2ceb281d-ec00-49c2-8434-b36273ac882e, 43, Finished, Available)

In [42]:
display(sdf.select(['first_review', 'last_review', 'host_since']))

StatementMeta(, 2ceb281d-ec00-49c2-8434-b36273ac882e, 44, Finished, Available)

SynapseWidget(Synapse.DataFrame, 6e8ec509-f752-4d6a-9902-5f638179a651)

In [43]:
# I decided to drop first_review and last_review because of the amount of missing values

sdf = sdf.drop('first_review', 'last_review')

StatementMeta(, 2ceb281d-ec00-49c2-8434-b36273ac882e, 45, Finished, Available)

In [44]:
# Convert back to pandas dataframe

psdf = ps.DataFrame(sdf)

StatementMeta(, 2ceb281d-ec00-49c2-8434-b36273ac882e, 46, Finished, Available)

In [45]:
psdf['longitude'] = psdf['longitude'].astype(float)
psdf['latitude'] = psdf['latitude'].astype(float)

StatementMeta(, 2ceb281d-ec00-49c2-8434-b36273ac882e, 47, Finished, Available)

# Convert to polar coordinates

The use of polar coordinates "explicitly models interactions" in the sense that the transformation from Cartesian coordinates (x, y) to polar coordinates (r, θ) inherently captures and separates the magnitude and directional components of a point's position. Let's break this down

In [46]:
psdf['polar_r'] = np.sqrt(psdf['latitude']**2 + psdf['longitude']**2)
psdf['polar_theta'] = np.arctan2(psdf['latitude'], psdf['longitude'])


StatementMeta(, 2ceb281d-ec00-49c2-8434-b36273ac882e, 48, Finished, Available)

In [47]:
psdf.to_spark()

StatementMeta(, 2ceb281d-ec00-49c2-8434-b36273ac882e, 49, Finished, Available)

DataFrame[accommodates: bigint, availability_30: bigint, availability_365: bigint, availability_60: bigint, availability_90: bigint, bathrooms: double, bedrooms: bigint, beds: bigint, extra_people: bigint, features: string, guests_included: bigint, host_listings_count: bigint, host_response_rate: bigint, host_since: int, host_total_listings_count: bigint, host_verifications: string, latitude: double, longitude: double, maximum_nights: bigint, minimum_nights: bigint, number_of_reviews: bigint, price: bigint, review_scores_accuracy: bigint, review_scores_checkin: bigint, review_scores_cleanliness: bigint, review_scores_communication: bigint, review_scores_location: bigint, review_scores_rating: bigint, review_scores_value: bigint, reviews_per_month: double, bed_type_Airbed: tinyint, bed_type_Couch: tinyint, bed_type_Futon: tinyint, bed_type_Pull-out Sofa: tinyint, bed_type_Real Bed: tinyint, cancellation_policy_flexible: tinyint, cancellation_policy_flexible_new: tinyint, cancellation_po

In [48]:
# Remove all the columns that are still note encoded
string_columns = psdf.select_dtypes(include=['object']).columns.tolist()
psdf = psdf.drop(columns=string_columns)

StatementMeta(, 2ceb281d-ec00-49c2-8434-b36273ac882e, 50, Finished, Available)

In [49]:
psdf = psdf.dropna(subset=['price'])

StatementMeta(, 2ceb281d-ec00-49c2-8434-b36273ac882e, 51, Finished, Available)

In [50]:
print(len(psdf.dropna()))

StatementMeta(, 2ceb281d-ec00-49c2-8434-b36273ac882e, 52, Finished, Available)

111128


## EDA conclusions

Based on additional EDA we have came to the following conclusions:

- Remove review score variables
- Remove variables -> number of beds
- Remove avaialbility60 & avaiability90
- Remove reviews per month and keep total number of reviews 

In [51]:
# Remove variables

columns_to_drop = ['review_scores_accuracy',
       'review_scores_checkin', 'review_scores_cleanliness',
       'review_scores_communication', 'review_scores_location',
       'review_scores_rating', 'review_scores_value', 'reviews_per_month',
       'availability_60', 'availability_90', 'beds', 'longitude', 'latitude']

psdf = psdf.drop(columns_to_drop, axis = 1)

StatementMeta(, 2ceb281d-ec00-49c2-8434-b36273ac882e, 53, Finished, Available)

**We will not yet apply imputation, we will commit a seperate notebook fir this purpose**

### Write to delta

Parquet formatted files do not accept column names with spaces or the following characters:

- '-'
- '&'
- '/'
- '('
- ')'

. At the moment there are spaces in the columns. Let's rename columns by replacing space by '_'


In [58]:
psdf.columns = [col.replace(" ", "_")\
                   .replace("-", "_")\
                   .replace("&", "and")\
                   .replace("/",  "_or_", )\
                   .replace("(", "")\
                   .replace(")", "")\
                    for col in psdf.columns]

StatementMeta(, 2ceb281d-ec00-49c2-8434-b36273ac882e, 60, Finished, Available)

In [59]:
psdf.columns

StatementMeta(, 2ceb281d-ec00-49c2-8434-b36273ac882e, 61, Finished, Available)

Index(['accommodates', 'availability_30', 'availability_365', 'bathrooms',
       'bedrooms', 'extra_people', 'guests_included', 'host_listings_count',
       'host_response_rate', 'host_since', 'host_total_listings_count',
       'maximum_nights', 'minimum_nights', 'number_of_reviews', 'price',
       'bed_type_Airbed', 'bed_type_Couch', 'bed_type_Futon',
       'bed_type_Pull_out_Sofa', 'bed_type_Real_Bed',
       'cancellation_policy_flexible', 'cancellation_policy_flexible_new',
       'cancellation_policy_moderate', 'cancellation_policy_moderate_new',
       'cancellation_policy_strict', 'cancellation_policy_strict_new',
       'cancellation_policy_super_strict_30',
       'cancellation_policy_super_strict_30_new',
       'cancellation_policy_super_strict_60',
       'cancellation_policy_super_strict_60_new',
       'host_response_time_a_few_days_or_more',
       'host_response_time_within_a_day',
       'host_response_time_within_a_few_hours',
       'host_response_time_within_an

In [60]:
sdf_final = psdf.dropna().to_spark()

StatementMeta(, 2ceb281d-ec00-49c2-8434-b36273ac882e, 62, Finished, Available)

/opt/spark/python/lib/pyspark.zip/pyspark/pandas/utils.py:975: PandasAPIOnSparkAdviceWarning: If `index_col` is not specified for `to_spark`, the existing index is lost when converting to Spark DataFrame.
  warnings.warn(message, PandasAPIOnSparkAdviceWarning)


In [61]:
sdf_final.write.format( "delta").option("mergeSchema", "true")\
               .mode("overwrite")\
               .save("Files/transformed/opendatasoft/airbnb-listings")

StatementMeta(, 2ceb281d-ec00-49c2-8434-b36273ac882e, 63, Finished, Available)